In [ ]:
#@title Dependencies
!pip install -q openai
!pip install -q gradio
!pip install -q graphwiz

In [ ]:
#@title Graphwiz and Openai
import json
from openai import OpenAI
from graphviz import Digraph

OPENAI_API_KEY = "Magic goes here."
openai_client = OpenAI(api_key = OPENAI_API_KEY)

from graphviz import Digraph

def create_mind_map(mind_map_structure):
    # Initialize the graph with some global styles
    dot = Digraph(comment='Mind Map',
                  graph_attr={'rankdir': 'LR', 'bgcolor': 'transparent'},
                  node_attr={'shape': 'box', 'style': 'rounded,filled', 'fillcolor': 'lightblue1', 'color': 'darkblue', 'fontname': 'Helvetica'},
                  edge_attr={'color': 'grey', 'arrowhead': 'vee'})

    # Recursively create nodes and edges
    def add_nodes_edges(parent_name, children, level=0):
        for child in children:
            child_name = f"{parent_name}_{child['node']}_{level}"

            # Custom colors for different levels, if desired
            #fillcolor = f"{'/'.join(['white', str(level * 10)] + (['lightblue1'] * (9-level)))}"

            dot.node(child_name, child['node'], fontcolor='black')
            dot.edge(parent_name, child_name)

            if 'children' in child:
                add_nodes_edges(child_name, child['children'], level+1)

    root = mind_map_structure['node']
    dot.node(root, root, shape='ellipse')  # Custom color and shape for root

    add_nodes_edges(root, mind_map_structure.get('children', []))

    format = 'svg'
    # Render the mind map to a file called 'image.svg' with pretty formatting
    dot.render(filename='image', format=format, cleanup=True)

    return f'image.{format}'


def create_mind_map_no_styling(mind_map_structure):
    dot = Digraph(comment='Mind Map')

    # Recursively create nodes and edges
    def add_nodes_edges(parent_name, children):
        for child in children:
            # Create a unique name for the child
            child_name = f"{parent_name}_{child['node']}"

            # Add the child node to the graph
            dot.node(child_name, child['node'])

            # Add an edge between the parent and the child
            dot.edge(parent_name, child_name)

            # If the child has its own children, recurse
            if 'children' in child:
                add_nodes_edges(child_name, child['children'])

    # The root of the mind map
    root = mind_map_structure['node']
    dot.node(root, root)

    # Start the recursion
    add_nodes_edges(root, mind_map_structure.get('children', []))

    format = 'svg'

    # Render the mind map to a file (e.g., 'mindmap.pdf')
    dot.render('mindmap', format=format, cleanup=True)

    return f'mindmap.{format}'


# Example of a mind map structure
mind_map = {
    'node': 'Root Concept',
    'children': [
        {
            'node': 'Subconcept A',
            'children': [
                {'node': 'Detail A1'},
                {'node': 'Detail A2'}
            ]
        },
        {
            'node': 'Subconcept B',
            'children': [
                {'node': 'Detail B1'},
            ]
        },
        {
            'node': 'Subconcept C',
        }
    ]
}

system_prompt = """
You are a mind map creator assistant, who creates a Graphwiz mind map structure in JSON.
Follow the following structure:
'data' :
  '{
      'node': 'Root Concept',
      'children': [
          {
              'node': 'Subconcept A',
              'children': [
                  {'node': 'Detail A1'},
                  {'node': 'Detail A2'}
              ]
          },
          {
              'node': 'Subconcept B',
              'children': [
                  {'node': 'Detail B1'},
              ]
          },
          {
              'node': 'Subconcept C',
          }
      ]
  }'
"""

def respond_to_user(text):
  response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": text}
    ],
  )

  mind_map = response.choices[0].message.content
  mind_map = json.loads(mind_map)['data']
  print(mind_map)

  return create_mind_map(mind_map)



In [ ]:
#@title Lil test if all working.
print(mind_map)
create_mind_map(mind_map)

In [ ]:
#@title A gradio page to make a UI!
import gradio as gr

with gr.Blocks() as app:
  input_text = gr.Textbox(label="What would you like to make a mind map of?")
  submit_button = gr.Button("Submit")
  output_image = gr.Image()
  submit_button.click(respond_to_user, input_text, output_image)
  input_text.submit(respond_to_user, input_text, output_image)

app.css="footer {visibility: hidden}"
app.title="Mind Map Maker MMM"
app.queue()
app.launch()